# Testing of Hypothesis D

This is the hypothesis that message passing can improve edge, if not node, metric learning

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
from pprint import pprint as pp
from time import time as tt
import inspect
import logging
import copy

# External imports
import matplotlib.pyplot as plt
import matplotlib.colors
from sklearn.decomposition import PCA
from sklearn.metrics import auc, f1_score 
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from mpl_toolkits.mplot3d import Axes3D
from itertools import permutations
from itertools import chain

from torch.nn import Linear
from torch_scatter import scatter, segment_csr, scatter_add
from torch_geometric.nn.conv import MessagePassing
from torch_cluster import knn_graph, radius_graph
import trackml.dataset
import torch_geometric
import yaml

import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.loggers import WandbLogger
from torch.utils.checkpoint import checkpoint

# Limit CPU usage on Jupyter
os.environ['OMP_NUM_THREADS'] = '4'

# Pick up local packages
sys.path.append('..')

# Local imports
# from lightning_modules.GravNet.utils import evaluate_set_metrics, get_metrics, model_evaluation, build_edges, graph_intersection
from lightning_modules.Filter.utils import edge_model_evaluation
from pytorch_lightning.loggers import WandbLogger
%matplotlib inline

# Get rid of RuntimeWarnings, gross
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [3]:
logging.basicConfig(level=logging.INFO)

In [63]:
logging.basicConfig(level=logging.WARNING)

In [ ]:
trainer = Trainer(gpus=1, max_epochs=2, logger=wandb_logger, num_sanity_val_steps=0)

## GNNNodeEmbedding Performance

In [4]:
from lightning_modules.GNNEmbedding.Models.agnn import AttentionNodeEmbedding

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Loading faiss.


In [5]:
with open("../lightning_modules/GNNEmbedding/train_coda_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
model = AttentionNodeEmbedding(hparams)
wandb_logger = WandbLogger(project='End2End-GNNNodeEmbedding')
wandb_logger.watch(model)
trainer = Trainer(gpus=1, max_epochs=15, logger=wandb_logger, num_sanity_val_steps=0)

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: murnanedaniel (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


### Training

In [7]:
trainer.fit(model)

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021009
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021088
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021089
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021090
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021091
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021092
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021093
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021094
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021095
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021096
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021177
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021178
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021179
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021180
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021181
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021182
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021183
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021184
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021185
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021261
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021262
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021263
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021264
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021265
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021266
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021267
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021268
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021269
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021346
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021347
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021349
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021350
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021351
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021352
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021353
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021354
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021356
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

...

INFO:root:Efficiency: 0.8791521191596985
INFO:root:Purity: 0.20620308816432953
INFO:root:Efficiency: 0.8590011596679688
INFO:root:Purity: 0.20141473412513733
INFO:root:Efficiency: 0.828018844127655
INFO:root:Purity: 0.19016927480697632
INFO:root:Efficiency: 0.8739306926727295
INFO:root:Purity: 0.20350567996501923
INFO:root:Efficiency: 0.874768078327179
INFO:root:Purity: 0.20269246399402618
INFO:root:Efficiency: 0.8719743490219116
INFO:root:Purity: 0.2045687586069107
INFO:root:Efficiency: 0.8787786960601807
INFO:root:Purity: 0.2091692090034485
INFO:root:Efficiency: 0.879444420337677
INFO:root:Purity: 0.2045205980539322
INFO:root:Efficiency: 0.8433977365493774
INFO:root:Purity: 0.19533590972423553
INFO:root:Efficiency: 0.8789713978767395
INFO:root:Purity: 0.20583492517471313
INFO:root:Efficiency: 0.8751109838485718
INFO:root:Purity: 0.20159897208213806
INFO:root:Efficiency: 0.8254950046539307
INFO:root:Purity: 0.19073128700256348
INFO:root:Efficiency: 0.8166205286979675
INFO:root:Purity:

INFO:root:Efficiency: 0.8887089490890503
INFO:root:Purity: 0.21245060861110687
INFO:root:Efficiency: 0.8679918646812439
INFO:root:Purity: 0.20795516669750214
INFO:root:Efficiency: 0.8368238210678101
INFO:root:Purity: 0.19441115856170654
INFO:root:Efficiency: 0.8866211175918579
INFO:root:Purity: 0.2097063958644867
INFO:root:Efficiency: 0.8877010345458984
INFO:root:Purity: 0.2101026177406311
INFO:root:Efficiency: 0.8816159963607788
INFO:root:Purity: 0.21035340428352356
INFO:root:Efficiency: 0.8862080574035645
INFO:root:Purity: 0.21404387056827545
INFO:root:Efficiency: 0.8913210034370422
INFO:root:Purity: 0.2102578580379486
INFO:root:Efficiency: 0.8527396321296692
INFO:root:Purity: 0.19963544607162476
INFO:root:Efficiency: 0.8913624882698059
INFO:root:Purity: 0.21204489469528198
INFO:root:Efficiency: 0.885235071182251
INFO:root:Purity: 0.20800474286079407
INFO:root:Efficiency: 0.8349225521087646
INFO:root:Purity: 0.1961316019296646
INFO:root:Efficiency: 0.8249974250793457
INFO:root:Purity

INFO:root:Efficiency: 0.9015301465988159
INFO:root:Purity: 0.2170703262090683
INFO:root:Efficiency: 0.8785955309867859
INFO:root:Purity: 0.21293522417545319
INFO:root:Efficiency: 0.850400984287262
INFO:root:Purity: 0.19929112493991852
INFO:root:Efficiency: 0.8936601877212524
INFO:root:Purity: 0.2138671576976776
INFO:root:Efficiency: 0.8970423340797424
INFO:root:Purity: 0.21422085165977478
INFO:root:Efficiency: 0.8960261940956116
INFO:root:Purity: 0.21636462211608887
INFO:root:Efficiency: 0.8983237743377686
INFO:root:Purity: 0.2186446338891983
INFO:root:Efficiency: 0.9008672833442688
INFO:root:Purity: 0.21368612349033356
INFO:root:Efficiency: 0.8637464642524719
INFO:root:Purity: 0.2038598507642746
INFO:root:Efficiency: 0.8999568223953247
INFO:root:Purity: 0.21603438258171082
INFO:root:Efficiency: 0.8959383368492126
INFO:root:Purity: 0.21160463988780975
INFO:root:Efficiency: 0.8482552170753479
INFO:root:Purity: 0.2014165073633194
INFO:root:Efficiency: 0.840412974357605
INFO:root:Purity: 

INFO:root:Efficiency: 0.908100962638855
INFO:root:Purity: 0.22339923679828644
INFO:root:Efficiency: 0.8836434483528137
INFO:root:Purity: 0.21789905428886414
INFO:root:Efficiency: 0.8590694069862366
INFO:root:Purity: 0.20414306223392487
INFO:root:Efficiency: 0.8973514437675476
INFO:root:Purity: 0.21917760372161865
INFO:root:Efficiency: 0.9056736826896667
INFO:root:Purity: 0.22198791801929474
INFO:root:Efficiency: 0.9007111191749573
INFO:root:Purity: 0.22123096883296967
INFO:root:Efficiency: 0.9037858843803406
INFO:root:Purity: 0.2234938144683838
INFO:root:Efficiency: 0.9050453305244446
INFO:root:Purity: 0.21790961921215057
INFO:root:Efficiency: 0.8669129014015198
INFO:root:Purity: 0.20819653570652008
INFO:root:Efficiency: 0.9035896062850952
INFO:root:Purity: 0.21924981474876404
INFO:root:Efficiency: 0.9025648832321167
INFO:root:Purity: 0.2184704691171646
INFO:root:Efficiency: 0.8548436760902405
INFO:root:Purity: 0.20643176138401031
INFO:root:Efficiency: 0.8445230722427368
INFO:root:Puri

1

In [8]:
trainer.test(ckpt_path=None)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


INFO:root:Efficiency: 0.9606665372848511
INFO:root:Purity: 0.042455099523067474
INFO:root:Efficiency: 0.9483796954154968
INFO:root:Purity: 0.04454655945301056
INFO:root:Efficiency: 0.9354404807090759
INFO:root:Purity: 0.04235435649752617
INFO:root:Efficiency: 0.9662193655967712
INFO:root:Purity: 0.04681840166449547
INFO:root:Efficiency: 0.9494401812553406
INFO:root:Purity: 0.042013052850961685
INFO:root:Efficiency: 0.9606983065605164
INFO:root:Purity: 0.04424251988530159
INFO:root:Efficiency: 0.9492781162261963
INFO:root:Purity: 0.0424494706094265
INFO:root:Efficiency: 0.9806956648826599
INFO:root:Purity: 0.04888773709535599
INFO:root:Efficiency: 0.9418500661849976
INFO:root:Purity: 0.0421764999628067
INFO:root:Efficiency: 0.9675005078315735
INFO:root:Purity: 0.04380270093679428
INFO:root:Efficiency: 0.9529621601104736
INFO:root:Purity: 0.04328588768839836
INFO:root:Efficiency: 0.9428498148918152
INFO:root:Purity: 0.04192404821515083
INFO:root:Efficiency: 0.9265901446342468
INFO:root:P

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'edge_eff': tensor(0.9470, device='cuda:0'),
 'edge_pur': tensor(0.8949, device='cuda:0'),
 'loss': tensor(0.9292, device='cuda:0'),
 'truth': array([False, False,  True, ..., False, False, False]),
 'truth_graph': array([[ 4096,  4096,  4096, ...,  4564,  4564,  4564],
       [12525,  5488, 13059, ..., 11186,   489,   538]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'edge_eff': tensor(0.9517, device='cuda:0'),
 'edge_pur': tensor(0.8919, device='cuda:0'),
 'loss': tensor(0.9184, device='cuda:0'),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[11801, 11801, 11801, ...,  4607,  4607,  4607],
       [ 4250,  1583, 12892, ..., 11526,  5371,  9663]])}
--------------------------------------------------------------------------------
DATALOADER:2 TEST RESULTS
{'edge_eff': tensor

[{'loss': 0.9292374849319458,
  'truth': array([False, False,  True, ..., False, False, False]),
  'truth_graph': array([[ 4096,  4096,  4096, ...,  4564,  4564,  4564],
         [12525,  5488, 13059, ..., 11186,   489,   538]]),
  'edge_eff': 0.946976363658905,
  'edge_pur': 0.8948776721954346},
 {'loss': 0.9184256792068481,
  'truth': array([False, False, False, ..., False, False, False]),
  'truth_graph': array([[11801, 11801, 11801, ...,  4607,  4607,  4607],
         [ 4250,  1583, 12892, ..., 11526,  5371,  9663]]),
  'edge_eff': 0.9516987204551697,
  'edge_pur': 0.8918918967247009},
 {'loss': 0.9415866136550903,
  'truth': array([False, False, False, ..., False, False, False]),
  'truth_graph': array([[14466, 14466, 14466, ..., 13955, 13955, 13955],
         [ 5280,  5264, 13247, ..., 10639, 10670,  9188]]),
  'edge_eff': 0.9396825432777405,
  'edge_pur': 0.8860480189323425},
 {'loss': 0.8487526774406433,
  'truth': array([ True,  True, False, ..., False, False, False]),
  'trut

### Testing

In [14]:
from lightning_modules.utils import embedding_model_evaluation

In [15]:
embedding_model_evaluation(model, trainer, "eff", 0.98)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


INFO:root:Efficiency: 0.9934486746788025
INFO:root:Purity: 0.029454803094267845
INFO:root:Efficiency: 0.9892996549606323
INFO:root:Purity: 0.03239157795906067
INFO:root:Efficiency: 0.9825626611709595
INFO:root:Purity: 0.029854673892259598
INFO:root:Efficiency: 0.9944565892219543
INFO:root:Purity: 0.03517455235123634
INFO:root:Efficiency: 0.988997757434845
INFO:root:Purity: 0.02930307388305664
INFO:root:Efficiency: 0.9919276833534241
INFO:root:Purity: 0.03207845613360405
INFO:root:Efficiency: 0.9863601326942444
INFO:root:Purity: 0.029991282150149345
INFO:root:Efficiency: 0.9961959719657898
INFO:root:Purity: 0.03697479888796806
INFO:root:Efficiency: 0.9886592030525208
INFO:root:Purity: 0.030000928789377213
INFO:root:Efficiency: 0.9901608824729919
INFO:root:Purity: 0.032725438475608826
INFO:root:Efficiency: 0.9897274971008301
INFO:root:Purity: 0.030901741236448288
INFO:root:Efficiency: 0.9903450012207031
INFO:root:Purity: 0.029671864584088326
INFO:root:Efficiency: 0.9817708134651184
INFO:

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'edge_eff': tensor(0.9465, device='cuda:0'),
 'edge_pur': tensor(0.8820, device='cuda:0'),
 'loss': tensor(0.8624, device='cuda:0'),
 'truth': array([False,  True, False, ..., False, False, False]),
 'truth_graph': array([[ 4096,  4096,  4096, ...,  4564,  4564,  4564],
       [12525,  5488, 13059, ..., 11186,   489,   538]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'edge_eff': tensor(0.9527, device='cuda:0'),
 'edge_pur': tensor(0.8777, device='cuda:0'),
 'loss': tensor(0.8666, device='cuda:0'),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[11801, 11801, 11801, ...,  4607,  4607,  4607],
       [ 4250,  1583, 12892, ..., 11526,  5371,  9663]])}
--------------------------------------------------------------------------------
DATALOADER:2 TEST RESULTS
{'edge_eff': tensor

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.0314604980002712 0.9894915835840233


INFO:root:Efficiency: 0.9942396879196167
INFO:root:Purity: 0.026994144544005394
INFO:root:Efficiency: 0.9910747408866882
INFO:root:Purity: 0.028631648048758507
INFO:root:Efficiency: 0.9838752746582031
INFO:root:Purity: 0.02708592265844345


RuntimeError: Error in virtual void* faiss::gpu::StandardGpuResourcesImpl::allocMemory(const faiss::gpu::AllocRequest&) at /__w/faiss-wheels/faiss-wheels/faiss/faiss/gpu/StandardGpuResources.cpp:410: Error: 'err == cudaSuccess' failed: Failed to cudaMalloc 1610612736 bytes on device 0 (error 2 out of memory
Outstanding allocations:


## GNNEdgeEmbedding Performance

In [4]:
from lightning_modules.GNNEmbedding.Models.agnn import AttentionEdgeEmbedding

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Loading faiss.


In [5]:
with open("../lightning_modules/GNNEmbedding/train_coda_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
model = AttentionEdgeEmbedding(hparams)
wandb_logger = WandbLogger(project='End2End-GNNEdgeEmbedding')
wandb_logger.watch(model)
trainer = Trainer(gpus=1, max_epochs=15, logger=wandb_logger, num_sanity_val_steps=0)

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: murnanedaniel (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


### Training

In [7]:
trainer.fit(model)

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021009
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021088
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021089
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021090
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021091
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021092
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021093
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021094
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021095
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021096
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
INFO:root:Efficiency: 0.34959396719932556
INFO:root:Purity: 0.1084076538681984
INFO:root:Efficiency: 0.3086983263492584
INFO:root:Purity: 0.09072129428386688
INFO:root:Efficiency: 0.2778337895870209
INFO:root:Purity: 0.06759405136108398
INFO:root:Efficiency: 0.30151134729385376
INFO:root:Purity: 0.08857293426990509
INFO:root:Efficiency: 0.3032328486442566
INFO:root:Purity: 0.0941183865070343
INFO:root:Efficiency: 0.31754806637763977
INFO:root:Purity: 0.09886984527111053
INFO:root:Efficiency: 0.3288053274154663
INFO:root:Purity: 0.0995342805981636
INFO:root:Efficiency: 0.32358837127685547
INFO:root:Purity: 0.09515460580587387
INFO:root:Efficiency: 0.28601980209350586
INFO:root:Purity: 0.07041013240814209
INFO:root:Efficiency

INFO:root:Efficiency: 0.49242743849754333
INFO:root:Purity: 0.15269966423511505
INFO:root:Efficiency: 0.45072463154792786
INFO:root:Purity: 0.1324603408575058
INFO:root:Efficiency: 0.4246377646923065
INFO:root:Purity: 0.10330992192029953
INFO:root:Efficiency: 0.44678422808647156
INFO:root:Purity: 0.13124875724315643
INFO:root:Efficiency: 0.4396956264972687
INFO:root:Purity: 0.136474147439003
INFO:root:Efficiency: 0.459632009267807
INFO:root:Purity: 0.14310823380947113
INFO:root:Efficiency: 0.4727110266685486
INFO:root:Purity: 0.14309664070606232
INFO:root:Efficiency: 0.4614516496658325
INFO:root:Purity: 0.1356947273015976
INFO:root:Efficiency: 0.4309903085231781
INFO:root:Purity: 0.10609785467386246
INFO:root:Efficiency: 0.4795423448085785
INFO:root:Purity: 0.14796391129493713
INFO:root:Efficiency: 0.46182361245155334
INFO:root:Purity: 0.13264432549476624
INFO:root:Efficiency: 0.4519958198070526
INFO:root:Purity: 0.11590688675642014
INFO:root:Efficiency: 0.3994942903518677
INFO:root:Pu

INFO:root:Efficiency: 0.538434624671936
INFO:root:Purity: 0.16696636378765106
INFO:root:Efficiency: 0.5001672506332397
INFO:root:Purity: 0.1469898223876953
INFO:root:Efficiency: 0.47612109780311584
INFO:root:Purity: 0.1158352792263031
INFO:root:Efficiency: 0.48607292771339417
INFO:root:Purity: 0.14279015362262726
INFO:root:Efficiency: 0.48723194003105164
INFO:root:Purity: 0.1512286216020584
INFO:root:Efficiency: 0.5127094388008118
INFO:root:Purity: 0.15963411331176758
INFO:root:Efficiency: 0.5197347402572632
INFO:root:Purity: 0.1573314666748047
INFO:root:Efficiency: 0.5148665904998779
INFO:root:Purity: 0.15140201151371002
INFO:root:Efficiency: 0.48335155844688416
INFO:root:Purity: 0.11898773908615112
INFO:root:Efficiency: 0.5258579254150391
INFO:root:Purity: 0.1622546762228012
INFO:root:Efficiency: 0.5112963318824768
INFO:root:Purity: 0.14685380458831787
INFO:root:Efficiency: 0.5016536116600037
INFO:root:Purity: 0.128640815615654
INFO:root:Efficiency: 0.4533243477344513
INFO:root:Purit

/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

TODO:
Visualise purity/eff(?) around TRUE edges only, and FALSE edges only. I.e. Can this method be used to distinguish fake from true edges, or in tandem with filter to 

In [8]:
trainer.test(ckpt_path=None)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


INFO:root:Efficiency: 0.9277957677841187
INFO:root:Purity: 0.5653819441795349
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021010']
INFO:root:Efficiency: 0.9343464374542236
INFO:root:Purity: 0.5432947278022766
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021024']
INFO:root:Efficiency: 0.9319637417793274
INFO:root:Purity: 0.5591942667961121
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021038']
INFO:root:Efficiency: 0.9341900944709778
INFO:root:Purity: 0.701016902923584
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021082']
INFO:root:Efficiency: 0.9276788234710693
INFO:root:Purity: 0.590079665184021
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021135']
INFO:root:Efficiency: 0.9397476315498352
INFO:root:Purity: 0.5796317458152771
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/tra

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'doublet_truth': array([False,  True, False, ...,  True,  True,  True]),
 'loss': tensor(0.3595, device='cuda:0'),
 'preds': array([[     1,      2,      6, ..., 106077, 106077, 106077],
       [    14,     15,      9, ..., 104951, 103391, 103390]]),
 'truth': array([False, False, False, ...,  True,  True,  True]),
 'truth_graph': array([[35377, 35377, 35377, ..., 39070, 39070, 39070],
       [34201, 53971, 34565, ..., 48700, 48701, 48703]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'doublet_truth': array([False,  True,  True, ...,  True,  True, False]),
 'loss': tensor(0.3894, device='cuda:0'),
 'preds': array([[    2,     2,     2, ..., 86783, 86783, 86784],
       [ 7305,     7,  3016, ..., 83468, 83469, 86781]]),
 'truth': array([ True, False, False, ...,  True,  True, False]),
 'truth_graph': array([[85900, 8

[{'loss': 0.3594932556152344,
  'preds': array([[     1,      2,      6, ..., 106077, 106077, 106077],
         [    14,     15,      9, ..., 104951, 103391, 103390]]),
  'truth': array([False, False, False, ...,  True,  True,  True]),
  'truth_graph': array([[35377, 35377, 35377, ..., 39070, 39070, 39070],
         [34201, 53971, 34565, ..., 48700, 48701, 48703]]),
  'doublet_truth': array([False,  True, False, ...,  True,  True,  True])},
 {'loss': 0.3893519341945648,
  'preds': array([[    2,     2,     2, ..., 86783, 86783, 86784],
         [ 7305,     7,  3016, ..., 83468, 83469, 86781]]),
  'truth': array([ True, False, False, ...,  True,  True, False]),
  'truth_graph': array([[85900, 85900, 85900, ..., 77627, 77627, 77627],
         [84767, 84766, 83380, ..., 83730, 77624, 77623]]),
  'doublet_truth': array([False,  True,  True, ...,  True,  True, False])},
 {'loss': 0.3546825647354126,
  'preds': array([[     4,      4,      4, ..., 126891, 126891, 126891],
         [ 12907,  

### Testing

In [19]:
from lightning_modules.utils import embedding_model_evaluation

[autoreload of lightning_modules.EdgeEmbedding.Models.tight_filter_embedding failed: Traceback (most recent call last):
  File "/global/homes/d/danieltm/.local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/global/homes/d/danieltm/.local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../lightning_modules/EdgeEmbedding/Models/tight_f

In [21]:
embedding_model_evaluation(model, trainer, "eff", 0.99)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


INFO:root:Efficiency: 0.9597566723823547
INFO:root:Purity: 0.18260617554187775
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021010']
INFO:root:Efficiency: 0.9617206454277039
INFO:root:Purity: 0.19844481348991394
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021024']
INFO:root:Efficiency: 0.957740843296051
INFO:root:Purity: 0.17416676878929138
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021038']
INFO:root:Efficiency: 0.9638161063194275
INFO:root:Purity: 0.27521812915802
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021082']
INFO:root:Efficiency: 0.957286536693573
INFO:root:Purity: 0.20312811434268951
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021135']
INFO:root:Efficiency: 0.9659504294395447
INFO:root:Purity: 0.1854756474494934
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/t

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'doublet_truth': array([False,  True, False, ...,  True,  True,  True]),
 'filter_eff': tensor(0.9309, device='cuda:0'),
 'filter_pur': tensor(0.7412, device='cuda:0'),
 'loss': tensor(0.4704, device='cuda:0'),
 'preds': array([[     0,      0,      0, ..., 106077, 106077, 106077],
       [105210, 105242, 103794, ..., 101921, 104017,  29727]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[ 35377,  35377,  35377, ...,  39070,  39070,  39070],
       [ 79876, 101692,  79877, ...,  80884,  91799,  48703]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'doublet_truth': array([False,  True,  True, ...,  True,  True, False]),
 'filter_eff': tensor(0.9308, device='cuda:0'),
 'filter_pur': tensor(0.7632, device='cuda:0'),
 'loss': tensor(0.5091, device='cuda:0'),
 'preds': array([[

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


,
 'loss': tensor(0.5231, device='cuda:0'),
 'preds': array([[     1,      1,      1, ..., 115668, 115668, 115668],
       [ 12504,  27951,  72438, ...,  30825, 115449,  32503]]),
 'truth': array([ True,  True,  True, ..., False,  True, False]),
 'truth_graph': array([[114599, 114599, 114599, ...,  62959,  62959,  62959],
       [ 63784,  63786,  82486, ..., 111421,  70004,  51942]])}
--------------------------------------------------------------------------------
DATALOADER:17 TEST RESULTS
{'doublet_truth': array([ True, False, False, ...,  True, False,  True]),
 'filter_eff': tensor(0.9331, device='cuda:0'),
 'filter_pur': tensor(0.7579, device='cuda:0'),
 'loss': tensor(0.4795, device='cuda:0'),
 'preds': array([[     0,      0,      0, ..., 110635, 110635, 110635],
       [  6720,   2045,  33656, ..., 107919,  60150,  47734]]),
 'truth': array([False, False, False, ...,  True,  True, False]),
 'truth_graph': array([[106311, 106311, 106311, ...,  72695,  72695,  72695],
       [1063

...

{'doublet_truth': array([False, False, False, ...,  True,  True,  True]),
 'filter_eff': tensor(0.9278, device='cuda:0'),
 'filter_pur': tensor(0.7916, device='cuda:0'),
 'loss': tensor(0.1091, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 67259, 67259, 67259],
       [33184, 33185, 29533, ..., 47310, 21078, 62005]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[41278, 41278, 41278, ..., 15829, 15829, 15829],
       [24539, 30030, 24540, ..., 23756, 23755, 63729]])}
--------------------------------------------------------------------------------
DATALOADER:14 TEST RESULTS
{'doublet_truth': array([False, False,  True, ...,  True, False,  True]),
 'filter_eff': tensor(0.9079, device='cuda:0'),
 'filter_pur': tensor(0.7059, device='cuda:0'),
 'loss': tensor(0.1183, device='cuda:0'),
 'preds': array([[     0,      0,      0, ..., 114691, 114691, 114691],
       [     1,  18698,   5243, ...,  79386,  40918, 114020]]),
 'truth': ar

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.04480375271180539 0.9891983884478543


INFO:root:Efficiency: 0.9882262349128723
INFO:root:Purity: 0.041108135133981705
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021010']


KeyboardInterrupt: 

## INEdgeEmbedding Performance

In [4]:
from lightning_modules.GNNEmbedding.Models.interaction_gnn import InteractionEdgeEmbedding

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Loading faiss.


In [5]:
with open("../lightning_modules/GNNEmbedding/train_coda_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
model = InteractionEdgeEmbedding(hparams)
wandb_logger = WandbLogger(project='End2End-GNNEdgeEmbedding')
wandb_logger.watch(model)
trainer = Trainer(gpus=1, max_epochs=20, logger=wandb_logger, num_sanity_val_steps=0)

ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: murnanedaniel (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


### Training

In [ ]:
trainer.fit(model)

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021000
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021001
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021002
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021004
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021005
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021006
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021007
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021008
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021009
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021088
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021089
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021090
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021091
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021092
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021093
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021094
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021095
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021096
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021177
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021178
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021179
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021180
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021181
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021182
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021183
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021184
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021185
INFO:root:Loaded event: /global/cscratch1/sd/danieltm/ExaTrkX/trackml-cod

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.

  | Name                   | Type           | Params
----------------------------------------------------------
0 | node_encoder           | Sequential     | 67 K  
1 | edge_encoder           | Sequential     | 197 K 
2 | edge_network           | Sequential     | 263 K 
3 | node_network           | Sequential     | 198 K 
4 | output_edge_classifier | Sequential     | 265 K 
5 | multi_loss             | MultiNoiseLoss | 0     
INFO:lightning:
  | Name                   | Type           | Params
----------------------------------------------------------
0 | node_encoder           | Sequential     | 67 K  
1 | edge_encoder           | Sequential     | 197 K 
2 | edge_network           | Sequential     | 263 K 
3 | node_network           | Sequential     | 198 K 
4 | output_edge_classifier | Sequential     | 265 K 
5 | multi_loss             | MultiNoiseLoss | 0     
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/py

/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
INFO:root:Efficiency: 0.7208735346794128
INFO:root:Purity: 0.11214127391576767
INFO:root:Efficiency: 0.6891089081764221
INFO:root:Purity: 0.10163114964962006
INFO:root:Efficiency: 0.6570258736610413
INFO:root:Purity: 0.08021768927574158
INFO:root:Efficiency: 0.6796042919158936
INFO:root:Purity: 0.10008388012647629
INFO:root:Efficiency: 0.6866344213485718
INFO:root:Purity: 0.10692794620990753
INFO:root:Efficiency: 0.6910287141799927
INFO:root:Purity: 0.107

INFO:root:Efficiency: 0.768750011920929
INFO:root:Purity: 0.12311359494924545
INFO:root:Efficiency: 0.7415057420730591
INFO:root:Purity: 0.11307711154222488
INFO:root:Efficiency: 0.7065582871437073
INFO:root:Purity: 0.09032917767763138
INFO:root:Efficiency: 0.7251571416854858
INFO:root:Purity: 0.10980166494846344
INFO:root:Efficiency: 0.7444149851799011
INFO:root:Purity: 0.11910950392484665
INFO:root:Efficiency: 0.7386825680732727
INFO:root:Purity: 0.11810463666915894
INFO:root:Efficiency: 0.7447143197059631
INFO:root:Purity: 0.11699333041906357
INFO:root:Efficiency: 0.7492226362228394
INFO:root:Purity: 0.1145685538649559
INFO:root:Efficiency: 0.7136843204498291
INFO:root:Purity: 0.09118110686540604
INFO:root:Efficiency: 0.7700734734535217
INFO:root:Purity: 0.12389633059501648
INFO:root:Efficiency: 0.7519726157188416
INFO:root:Purity: 0.11175466328859329
INFO:root:Efficiency: 0.7288231253623962
INFO:root:Purity: 0.09684935212135315
INFO:root:Efficiency: 0.6859443783760071
INFO:root:Pur

INFO:root:Efficiency: 0.7906656861305237
INFO:root:Purity: 0.13304471969604492
INFO:root:Efficiency: 0.7613814473152161
INFO:root:Purity: 0.12120942771434784
INFO:root:Efficiency: 0.730742335319519
INFO:root:Purity: 0.0981949046254158
INFO:root:Efficiency: 0.7507776021957397
INFO:root:Purity: 0.11814112961292267
INFO:root:Efficiency: 0.7630633115768433
INFO:root:Purity: 0.1268407553434372
INFO:root:Efficiency: 0.7646526098251343
INFO:root:Purity: 0.12736453115940094
INFO:root:Efficiency: 0.7735822796821594
INFO:root:Purity: 0.12753356993198395
INFO:root:Efficiency: 0.7717242240905762
INFO:root:Purity: 0.12370076775550842
INFO:root:Efficiency: 0.7377434968948364
INFO:root:Purity: 0.09835185110569
INFO:root:Efficiency: 0.7840206623077393
INFO:root:Purity: 0.13181129097938538
INFO:root:Efficiency: 0.7735140323638916
INFO:root:Purity: 0.12047187983989716
INFO:root:Efficiency: 0.749396026134491
INFO:root:Purity: 0.10391698032617569
INFO:root:Efficiency: 0.7118608951568604
INFO:root:Purity: 

TODO:
Visualise purity/eff(?) around TRUE edges only, and FALSE edges only. I.e. Can this method be used to distinguish fake from true edges, or in tandem with filter to 

In [18]:
trainer.test(ckpt_path=None)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.
/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


RuntimeError: CUDA out of memory. Tried to allocate 7.56 GiB (GPU 0; 15.78 GiB total capacity; 8.46 GiB already allocated; 5.85 GiB free; 8.67 GiB reserved in total by PyTorch)

### Testing

In [19]:
from lightning_modules.utils import embedding_model_evaluation

[autoreload of lightning_modules.EdgeEmbedding.Models.tight_filter_embedding failed: Traceback (most recent call last):
  File "/global/homes/d/danieltm/.local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/global/homes/d/danieltm/.local/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/global/homes/d/danieltm/.conda/envs/exatrkx-test/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../lightning_modules/EdgeEmbedding/Models/tight_f

In [21]:
embedding_model_evaluation(model, trainer, "eff", 0.99)

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


INFO:root:Efficiency: 0.9597566723823547
INFO:root:Purity: 0.18260617554187775
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021010']
INFO:root:Efficiency: 0.9617206454277039
INFO:root:Purity: 0.19844481348991394
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021024']
INFO:root:Efficiency: 0.957740843296051
INFO:root:Purity: 0.17416676878929138
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021038']
INFO:root:Efficiency: 0.9638161063194275
INFO:root:Purity: 0.27521812915802
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021082']
INFO:root:Efficiency: 0.957286536693573
INFO:root:Purity: 0.20312811434268951
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021135']
INFO:root:Efficiency: 0.9659504294395447
INFO:root:Purity: 0.1854756474494934
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/t

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'doublet_truth': array([False,  True, False, ...,  True,  True,  True]),
 'filter_eff': tensor(0.9309, device='cuda:0'),
 'filter_pur': tensor(0.7412, device='cuda:0'),
 'loss': tensor(0.4704, device='cuda:0'),
 'preds': array([[     0,      0,      0, ..., 106077, 106077, 106077],
       [105210, 105242, 103794, ..., 101921, 104017,  29727]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[ 35377,  35377,  35377, ...,  39070,  39070,  39070],
       [ 79876, 101692,  79877, ...,  80884,  91799,  48703]])}
--------------------------------------------------------------------------------
DATALOADER:1 TEST RESULTS
{'doublet_truth': array([False,  True,  True, ...,  True,  True, False]),
 'filter_eff': tensor(0.9308, device='cuda:0'),
 'filter_pur': tensor(0.7632, device='cuda:0'),
 'loss': tensor(0.5091, device='cuda:0'),
 'preds': array([[

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


,
 'loss': tensor(0.5231, device='cuda:0'),
 'preds': array([[     1,      1,      1, ..., 115668, 115668, 115668],
       [ 12504,  27951,  72438, ...,  30825, 115449,  32503]]),
 'truth': array([ True,  True,  True, ..., False,  True, False]),
 'truth_graph': array([[114599, 114599, 114599, ...,  62959,  62959,  62959],
       [ 63784,  63786,  82486, ..., 111421,  70004,  51942]])}
--------------------------------------------------------------------------------
DATALOADER:17 TEST RESULTS
{'doublet_truth': array([ True, False, False, ...,  True, False,  True]),
 'filter_eff': tensor(0.9331, device='cuda:0'),
 'filter_pur': tensor(0.7579, device='cuda:0'),
 'loss': tensor(0.4795, device='cuda:0'),
 'preds': array([[     0,      0,      0, ..., 110635, 110635, 110635],
       [  6720,   2045,  33656, ..., 107919,  60150,  47734]]),
 'truth': array([False, False, False, ...,  True,  True, False]),
 'truth_graph': array([[106311, 106311, 106311, ...,  72695,  72695,  72695],
       [1063

...

{'doublet_truth': array([False, False, False, ...,  True,  True,  True]),
 'filter_eff': tensor(0.9278, device='cuda:0'),
 'filter_pur': tensor(0.7916, device='cuda:0'),
 'loss': tensor(0.1091, device='cuda:0'),
 'preds': array([[    0,     0,     0, ..., 67259, 67259, 67259],
       [33184, 33185, 29533, ..., 47310, 21078, 62005]]),
 'truth': array([False, False, False, ..., False, False, False]),
 'truth_graph': array([[41278, 41278, 41278, ..., 15829, 15829, 15829],
       [24539, 30030, 24540, ..., 23756, 23755, 63729]])}
--------------------------------------------------------------------------------
DATALOADER:14 TEST RESULTS
{'doublet_truth': array([False, False,  True, ...,  True, False,  True]),
 'filter_eff': tensor(0.9079, device='cuda:0'),
 'filter_pur': tensor(0.7059, device='cuda:0'),
 'loss': tensor(0.1183, device='cuda:0'),
 'preds': array([[     0,      0,      0, ..., 114691, 114691, 114691],
       [     1,  18698,   5243, ...,  79386,  40918, 114020]]),
 'truth': ar

Set SLURM handle signals.
INFO:lightning:Set SLURM handle signals.


0.04480375271180539 0.9891983884478543


INFO:root:Efficiency: 0.9882262349128723
INFO:root:Purity: 0.041108135133981705
INFO:root:['/global/cscratch1/sd/danieltm/ExaTrkX/trackml-codalab/train_all/event000021010']


KeyboardInterrupt: 